In [1]:
import pandas as pd
from tqdm import tqdm
import json

df = pd.read_csv("./last_buy.csv")

out_file_name = "bert4rec.default.without_last_cls.config.epoch=13-step=104832"
out = json.load(open(f"../out/{out_file_name}.json", "r"))

df

,customer,order_date,product
0,00001f06-ae9a-4063-a22d-fd7754c89d74,2021-08-06 20:42:17,eede7de4-97b0-4f92-890d-599550b4d239
1,00002857-64cb-4112-afb9-aa8e6eb79eb4,2021-11-01 08:35:09,fb3ad737-237d-402e-9690-dddc9e03e173
2,0000cc39-a123-425f-b133-ae254f905176,2021-04-28 12:55:47,"2811ab88-c28a-40fc-8bc9-b38bcc5f1b25,1dfcd24d-..."
3,0001ae29-bf87-411e-afed-3545f835ac52,2021-11-30 09:53:53,64ed63da-7b5f-408e-a15c-6b494455963b
4,0001cd9d-eae6-41df-ae53-063592b30c67,2020-10-23 02:18:53,"53d4d05e-4eda-4fa1-9e24-598a7e812fb8,abad439e-..."
...,...,...,...
204093,ffff742d-a685-4161-bc54-4b4dac33ec39,2021-11-14 23:08:09,3feb364a-8d30-4dda-8fd0-c3b7d3a93707
204094,ffff7ef9-9a24-4368-8ac5-354aacd765f3,2020-10-17 22:58:21,"415365fe-0831-4eb8-ad42-bb1883bd87f3,cf4c2990-..."
204095,ffffecab-6e82-4ada-a55e-a104f2f899f6,2020-10-04 21:26:28,"e4a7f3a7-0cf7-44bb-8c86-b15b89f93406,0c995c04-..."
204096,fffff859-d3e9-4e6f-840a-5112618a3b77,2021-07-20 08:32:11,dba2f638-830e-4418-9e39-cf732127af66


In [2]:
def calculate_recall_precision_f1(dataframe: pd.DataFrame, pred_dict):
    user_history_dict = pd.Series(dataframe["product"].values,index=dataframe["customer"]).to_dict()
    hits = 0
    recall = 0
    precision = 0
    for u in tqdm(pred_dict):
        trues = user_history_dict[u].split(",")
        
        preds = pred_dict[u]
        hit = list(set(preds) & set(trues))    
        hits += len(hit)
        
        recall += len(hit)/len(trues)
        precision += len(hit)/len(preds)

    recall = recall/len(dataframe)
    precision = precision/len(dataframe)
    f1 = 2 * (precision * recall) / (precision + recall)

    
    return recall, precision, f1
        

In [3]:

recall, precision, f1 = calculate_recall_precision_f1(df, out)
print("total_recall", recall)
print("total_precision", precision)
print("total_f1", f1)

100%|██████████| 179701/179701 [00:00<00:00, 299308.80it/s]

total_recall 0.1461977745431492
total_precision 0.01608394006799743
total_f1 0.02897968202346192


In [4]:
out_file_name = "tva.default.without_last_cls.config.epoch=9-step=74880"

out2 = json.load(open(f"../out/{out_file_name}.json", "r"))
recall, precision, f1 = calculate_recall_precision_f1(df, out2)
print("total_recall", recall)
print("total_precision", precision)
print("total_f1", f1)

100%|██████████| 179701/179701 [00:00<00:00, 551246.63it/s]

total_recall 0.045250897696267584
total_precision 0.026475516663552662
total_f1 0.033405849342703756


In [5]:
out_file_name = "bert4rec.default.without_last_cls.config.epoch=9-step=74880"
out = json.load(open(f"../out/{out_file_name}.json", "r"))
print("predict_users", len(out))
recall, precision, f1 = calculate_recall_precision_f1(df, out)
print("total_recall", recall)
print("total_precision", precision)
print("total_f1", f1)

predict_users 179701


100%|██████████| 179701/179701 [00:00<00:00, 561799.31it/s]

total_recall 0.06098694183246808
total_precision 0.036987133631863324
total_f1 0.046047531587626936


In [70]:
out_file_name = "vaecf.vae_matrix.without_last_cls.config.epoch=9-step=25520"
out = json.load(open(f"../out/{out_file_name}.json", "r"))
print("predict_users", len(out))
recall, precision, f1 = calculate_recall_precision_f1(df, out)
print("total_recall", recall)
print("total_precision", precision)
print("total_f1", f1)

predict_users 204098


100%|██████████| 204098/204098 [00:00<00:00, 573979.09it/s]

total_recall 0.14249292488235613
total_precision 0.05782810218622946
total_f1 0.08226890148771429
